In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [5]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [6]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

52 documents
16 classes ['5g', 'ac', 'delivery', 'fridge', 'goodbye', 'greeting', 'hours', 'items', 'offers', 'online', 'payments', 'phones', 'screensize', 'thanks', 'tv', 'washingmachine']
95 unique lemmatized words ["'s", '5g', 'a', 'about', 'ac', 'accept', 'any', 'anyone', 'are', 'book', 'bye', 'can', 'card', 'cash', 'credit', 'delivery', 'do', 'doe', 'door', 'double', 'fridge', 'front', 'get', 'goodbye', 'have', 'hd', 'hello', 'helpful', 'hey', 'hi', 'hour', 'how', 'i', 'in', 'inverter', 'is', 'item', 'kind', 'later', 'latest', 'load', 'long', 'lot', 'machine', 'make', 'mastercard', 'maximum', 'me', 'model', 'my', 'of', 'offer', 'online', 'only', 'open', 'opening', 'order', 'pay', 'paypal', 'phone', 'product', 'qled', 'refridgerator', 'samsung', 'screen', 'screensize', 'see', 'sell', 'semi-automatic', 'shall', 'shipping', 'shop', 'single', 'size', 'special', 'split', 'store', 'take', 'tell', 'thank', 'thanks', 'that', 'the', 'there', 'time', 'top', 'tv', 'washing', 'washingmachine'

In [7]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        # output is a '0' for each tag and '1' for current tag (for each pattern)
        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1
        training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-7-cd2d2c45d8a7>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [8]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=12, batch_size=100, verbose=1)
print("model created")

Epoch 1/12
4940/4940 [==============================] - 431s 87ms/step - loss: 2.6720 - accuracy: 0.1441
Epoch 2/12
4940/4940 [==============================] - 402s 81ms/step - loss: 2.2522 - accuracy: 0.3652
Epoch 3/12
4940/4940 [==============================] - 464s 94ms/step - loss: 1.7100 - accuracy: 0.5640
Epoch 4/12
4940/4940 [==============================] - 380s 77ms/step - loss: 1.1853 - accuracy: 0.7115
Epoch 5/12
4940/4940 [==============================] - 507s 103ms/step - loss: 0.8272 - accuracy: 0.8047
Epoch 6/12
4940/4940 [==============================] - 411s 83ms/step - loss: 0.5943 - accuracy: 0.8575
Epoch 7/12
4940/4940 [==============================] - 375s 76ms/step - loss: 0.4440 - accuracy: 0.9018
Epoch 8/12
4940/4940 [==============================] - 387s 78ms/step - loss: 0.3434 - accuracy: 0.9243
Epoch 9/12
4940/4940 [==============================] - 419s 85ms/step - loss: 0.2858 - accuracy: 0.9429
Epoch 10/12
4940/4940 [==============================]

In [9]:
model.save('chatbot_model.h5', hist)

In [10]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))